In [1]:
import numpy as np
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
import math
from data_partition import DistributedDataSet
from init_ipfs_nodes import create_ipfs_nodes
import ipfshttpclient
import piskg
import copy
import multiprocessing
from multiprocessing.pool import ThreadPool
from datetime import datetime
from datetime import timedelta
import json
import time
from web3 import Web3, HTTPProvider
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
# train = pd.read_csv("dataset/train.csv")
# test = pd.read_csv("dataset/test.csv")

In [3]:
CLIENTS = 8 #don't set more than 9 because of IPFS initialization script
CLIENT_GRAPH = {
    0: [1, 4],
    1: [0, 5, 2],
    2: [3, 1],
    3: [2, 4, 6],
    4: [3, 7, 0],
    5: [1],
    6: [3],
    7: [4]
}

# CLIENTS = 4 #don't set more than 9 because of IPFS initialization script
# CLIENT_GRAPH = {
#     0: [1, 2],
#     1: [0, 3],
#     2: [0, 3],
#     3: [1, 2],
# }

SEED = 0
ROUNDS = 5
BATCH_SIZE = 30 # prefereably set as a multiple of the number of unique classes in y

#supports only pandas dataframe for now. 
#input values should be labelled as x0, x1, etc. 
#output values should be labelled as y
DATASET = pd.read_csv("dataset/train.csv")


server = Web3(HTTPProvider('http://localhost:8545'))
CONTRACT_ADDRESS = server.toChecksumAddress("0xab4b39212bF5BE56Fe755a7562A62Af2f71e368B")
DEFAULT_ADDRESS  = server.toChecksumAddress(server.eth.accounts[0])
with open('../blkTrial/HelloWorld/build/contracts/Model.json') as f:
    CONTRACT_DATA = json.load(f)
CONTRACT_ABI = CONTRACT_DATA['abi']
BLKCHAIN_ACCOUNTS = server.eth.accounts

In [4]:
dd = DistributedDataSet(DATASET, SEED, BATCH_SIZE, CLIENTS)
df_di, test = dd.get_distributed_dataset(0.1, 'label')

In [5]:
create_ipfs_nodes(CLIENTS)

generating ED25519 keypair...done
peer identity: 12D3KooWHiHSXnWk5kTQgx2Z4VLzTD55abSrEeK4LjwMNCtKojEQ
initializing IPFS node at /home/hitesh/.ipfs_fl0
to get started, enter:

	ipfs cat /ipfs/QmQPeNsJPyVWPFDVHb77w8G42Fvo15z4bG2X8D2GhfbSXc/readme

removed /dnsaddr/bootstrap.libp2p.io/p2p/QmNnooDu7bfjPFoTZYxMNLWUQJyrVwtbZg5gBMjTezGAJN
removed /dnsaddr/bootstrap.libp2p.io/p2p/QmQCU2EcMqAqQPR2i9bChDtGNJchTbq5TbXJJ16u19uLTa
removed /dnsaddr/bootstrap.libp2p.io/p2p/QmbLHAnMoJPWSCR5Zhtx6BHJX9KiKNN6tpvbUcqanj75Nb
removed /dnsaddr/bootstrap.libp2p.io/p2p/QmcZf59bWwK5XFi76CZX8cbJ4BhTzzA3gU1ZjYZcYW3dwt
removed /ip4/104.131.131.82/tcp/4001/p2p/QmaCpDMGvV2BGHeYERUEnRQAwe3N8SzbUtfsmvsqQLuvuJ
removed /ip4/104.131.131.82/udp/4001/quic/p2p/QmaCpDMGvV2BGHeYERUEnRQAwe3N8SzbUtfsmvsqQLuvuJ
generating ED25519 keypair...done
peer identity: 12D3KooWRV9ttRdvEhTBEgdvMv25J3ZzZbERjfGNy37sPnL2NrQ3
initializing IPFS node at /home/hitesh/.ipfs_fl1
to get started, enter:

	ipfs cat /ipfs/QmQPeNsJPyVWPFDVHb77w8G42Fvo15

Swarm is limited to private network of peers with the swarm key
Swarm key fingerprint: 3ead9a90d1dfde98026e4b0be59916b8
Swarm listening on /ip4/127.0.0.1/tcp/4008
Swarm listening on /ip4/192.168.201.125/tcp/4008
Swarm listening on /ip6/::1/tcp/4008
Swarm listening on /p2p-circuit
Swarm announcing /ip4/127.0.0.1/tcp/4008
Swarm announcing /ip4/192.168.201.125/tcp/4008
Swarm announcing /ip6/::1/tcp/4008
API server listening on /ip4/127.0.0.1/tcp/5008
WebUI: http://127.0.0.1:5008/webui
Gateway (readonly) server listening on /ip4/127.0.0.1/tcp/8087
Daemon is ready
Swarm is limited to private network of peers with the swarm key
Swarm key fingerprint: 3ead9a90d1dfde98026e4b0be59916b8
Swarm listening on /ip4/127.0.0.1/tcp/4005
Swarm listening on /ip4/192.168.201.125/tcp/4005
Swarm listening on /ip6/::1/tcp/4005
Swarm listening on /p2p-circuit
Swarm announcing /ip4/127.0.0.1/tcp/4005
Swarm announcing /ip4/192.168.201.125/tcp/4005
Swarm announcing /ip6/::1/tcp/4005
API server listening on /ip4/1

In [6]:
# class ClientNetwork:
#     def __init__(self, count, structure, ipfs, blkchain, train_data, test_data):
#         self.count = count
#         self.structure = structure
#         self.clients = ipfs
#         self.blkchain = blkchain
#         self.train_data = train_data
#         self.test_data = test_data
        
#         self.clients = []
        
#     def start_clients(self):
#         for i in range(self.count):
#             ag = Client(i, self.structure[i], sekf.train_data[i], self.ipfs, self.blkchain, self.test)
#             self.clients.append(ag)
            

In [7]:
def default(obj):
    if type(obj).__module__ == np.__name__:
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj.item()
    else:
        return obj.total_seconds()
#     raise TypeError('Unknown type:', type(obj))

In [8]:
def getTime(messages):
    simulated_communication_times = {i: messages[i]['time'] for i in range(len(messages))}
    slowest_client = max(simulated_communication_times, key=simulated_communication_times.get)
    simulated_time = simulated_communication_times[slowest_client]  # simulated time it would take for server to receive all values
    return simulated_time

In [9]:
class Message:
    def __init__(self, sender, receiver, body):
        self.sender = sender
        self.receiver = receiver
        self.body = body

In [10]:
class Client:
    def __init__(self, client_id, neighbour_ids, client_datasets, client_ipfs_node, client_blkchain_node, test_dataset):
        
        self.client_id = client_id
        self.neighbour_ids = neighbour_ids
        self.neighbour_ids.append(self.client_id) #neighbour includes itself
        self.client_datasets = client_datasets
        self.client_ipfs_node = client_ipfs_node
        self.client_blkchain_node = client_blkchain_node
        self.test_dataset = test_dataset
        
        self.personal_weights = {}
#         self.personal_intercepts = {}
        
        self.federated_weights = {}
#         self.federated_intercepts = {}
        
        self.personal_accuracy = {}
        self.federated_accuracy = {}
        
        self.train_time = {}
        
        self.nb_weights = {}
#         self.nb_intercepts = {}
        
        self.ipfsHash = {}
        
        self.nb_hash = {}
        
        self.blkchain_update = {}
    
    def addModelIpfs(self, round_it):
        start_time = datetime.now()
        model = {
                 'weights': self.personal_weights[round_it][len(self.client_datasets) - 1],
#                  'intercepts': self.personal_intercepts[round_it][len(self.client_datasets) - 1],
                 'time': self.train_time[round_it]
                }
        with open('model'+str(self.client_id)+'.json', 'w') as f:
            json.dump(model, f, default=default)

        cmd = "IPFS_PATH=~/"+self.client_ipfs_node+" ipfs add model"+str(self.client_id)+".json"
        
        try:
            modelHash = (os.popen(cmd).read()).strip()
            modelHash = modelHash.split()[1]
            self.ipfsHash[round_it] = modelHash
            stop_time = datetime.now()
            ipfs_time = (stop_time - start_time)
            return modelHash
        except:
            stop_time = datetime.now()
            ipfs_time = (stop_time - start_time)
            return 0
        
    def recModelIpfs(self, round_it, ipfs_hash):
#         try:
            cmd = "IPFS_PATH=~/"+self.client_ipfs_node+" ipfs cat "+ipfs_hash
            model = (os.popen(cmd).read()).strip()
            start_time = datetime.now()
            model = json.loads(os.popen(cmd).read())
            stop_time = datetime.now()
            ipfs_time = (stop_time - start_time)
            return model
#         except KeyError:
#             print(self.client_ipfs_node, round_it)
#             return None
        
    def modelChecker(self, round_it):
        try:
            a = self.ipfsHash[round_it]
            return None
        except KeyError:
            return self.client_id
    
    def client_modelCheck_caller(self, deets):
        client_instance, msg = deets
        round_it = msg.body['round']
        m = client_instance.modelChecker(round_it)
        return m
    
#     def client_model_caller(self, deets):
#         client_instance, msg = deets
#         round_it = msg.body['round']
#         m = client_instance.recModelIpfs(round_it)
#         return m

    def get_ipfs_hash(self, target_node_blk, round_it, target_node_id):
        contract = server.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)
        msg = contract.functions.getIpfsHashForUser(target_node_blk, str(round_it)).call()
        if(msg==''):
            print("Round "+str(round_it)+" missing on blockchain for Node "+str(target_node_id))
            return 0
        else:
            return (msg)
            

    def client_model_caller(self, deets):
        client_instance, msg = deets
        round_it = msg.body['round']
        target_node_blk = client_instance.client_blkchain_node
        target_node_id = client_instance.client_id
        ipfs_hash = self.get_ipfs_hash(target_node_blk, round_it, target_node_id)
        if(ipfs_hash!=0):
            m = self.recModelIpfs(round_it, ipfs_hash)
            return m
        else:
            print("Couldn't retreive ipfs hash from blockchain.")
    
    def client_train_caller(self, deets):
        client_instance, msg = deets
#         print("Training Caller: ", client_instance.client_id)
        if(client_instance.client_id == self.client_id):
            self.train_client(msg)
        else:  
            client_instance.train_client(msg)
#         print("Training Caller 2: ", client_instance.client_id)
    
    
    def network_train_caller(self, numRounds):
        for num in range(numRounds):
            print("Training round: ", num)
            self.get_nb_weights(num)
            time.sleep(1)
            print("Validation accuracy: ", self.personal_accuracy[num])
            print()
    
    def get_nb_weights(self, round_it):
        m = multiprocessing.Manager()
        lock = m.Lock()
        
        with ThreadPool(len(self.neighbour_ids)) as calling_pool:
            args = []
            for cl in (self.neighbour_ids):
                body = {'round': round_it, 'lock': lock}
                msg = Message(self.client_id, cl, body)
                args.append((allCl.clients[cl], msg))
            msgs = calling_pool.map(self.client_modelCheck_caller, args)
            msgs = np.array(msgs)
        
        untrained_nb = list(msgs[msgs!=None])
#         print("Current Node: ", self.client_id)
#         print("Untrained Neighbour: ", untrained_nb)
#         print("All neighbours: ", self.neighbour_ids)
        if(len(untrained_nb)!=0):
            with ThreadPool(len(untrained_nb)) as calling_pool:
                args = []
                for cl in (untrained_nb):
                    body = {'round': round_it, 'lock': lock}
                    msg = Message(self.client_id, cl, body)
                    args.append((allCl.clients[cl], msg))
                calling_pool.map(self.client_train_caller, args)
                
        with ThreadPool(len(self.neighbour_ids)) as calling_pool:
            args = []
            for cl in (self.neighbour_ids):
                body = {'round': round_it, 'lock': lock}
                msg = Message(self.client_id, cl, body)
                args.append((allCl.clients[cl], msg))
            msgs = calling_pool.map(self.client_model_caller, args)        
        
        time_start = datetime.now()
        train_time = getTime(msgs)
        
        self.nb_weights[round_it] = []
#         self.nb_intercepts[round_it] = []
        
        for msg in msgs:
            self.nb_weights[round_it].append(msg['weights'])
#             self.nb_intercepts[round_it].append(msg['intercepts'])
        
        time_stop = datetime.now()
        fedtime = time_stop - time_start
        totTime = fedtime+timedelta(train_time)
        self.FedAvg(round_it)
#         return msgs 
    
    def FedAvg(self, round_it):
        
        new_weights = list()
        for weights_list_tuple in zip(*self.nb_weights[round_it]):
            new_weights.append([np.array(weights_).mean(axis=0) for weights_ in zip(*weights_list_tuple)])

        
        self.federated_weights[round_it] = new_weights
#         self.federated_intercepts[round_it] = np.average(self.nb_intercepts[round_it], axis=0)
    
    def data_preprocessing(self, X, y):
        from keras.utils.np_utils import to_categorical
        X = X/255.0
        y = to_categorical(y, num_classes=10)
        X = X.values.reshape(-1, 28, 28, 1)
        return (X, y)
    
    def NN(self):
        
        import tensorflow as tf
        from keras.models import Sequential
        from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
        from tensorflow.keras.optimizers import RMSprop
        from keras.preprocessing.image import ImageDataGenerator
        from keras.callbacks import ReduceLROnPlateau
        
        tf.config.run_functions_eagerly(True)
        tf.data.experimental.enable_debug_mode()
        
        model = Sequential()

        model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                         activation ='relu', input_shape = (28,28,1)))
        model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                         activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2)))
        model.add(Dropout(0.25))


        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu'))
        model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                         activation ='relu'))
        model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
        model.add(Dropout(0.25))


        model.add(Flatten())
        model.add(Dense(256, activation = "relu"))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation = "softmax"))
        
        optimizer = RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
        model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"], run_eagerly=True)
        
        return model
        
    
    def train_client(self, message):
            training_flag = 0
            training_over = 0
#             print("Training node ", self.client_id)
            round_it = message.body['round']
            start_time = datetime.now()
            for epoch in range(len(self.client_datasets)):
                if(training_flag==0):
                    X = self.client_datasets[epoch].drop('y', axis=1)
                    y = self.client_datasets[epoch]['y']
                    
                    X, y = self.data_preprocessing(X, y)
                    
                    weights = self.compute_epoch(X, y, round_it, epoch, message.sender, self.client_id)

                    if(epoch==0):
                        try:
                            self.personal_weights[round_it]
                            training_flag = 1
                        except:
                            pass
                            self.personal_weights[round_it] = []
#                             self.personal_intercepts[round_it] = []

                    if(training_flag==0):
                        self.personal_weights[round_it].append(weights)
#                         self.personal_intercepts[round_it].append(intercepts)

            if(training_flag==0):
                stop_time = datetime.now()
                comp_time = stop_time - start_time
                self.train_time[round_it] = comp_time
                success = self.addModelIpfs(round_it)
#                 print("Training finished for ", self.client_id)
                if (success!=0):
                    contract = server.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)
                    tx_hash = contract.functions.addFile(str(round_it), success).transact({'from':self.client_blkchain_node})
                    rec = server.eth.waitForTransactionReceipt(tx_hash)
                    if(rec['status']!=1):
                        print('TRANSACTION REVERTED BY BLOCKCHAIN')
                    else:
                        self.blkchain_update[round_it] = 1
            else:
                while(training_over==0):
                    try:
                        self.blkchain_update[round_it]
                        training_over = 1
#                         print("Training over")
                    except:
                        time.sleep(1)
                        
#                 print("Client ", self.client_id, " training already.")
            
            
                    
    
    def compute_epoch(self, X, y, round_it, epoch, sender, receiver):            
        
        model = self.NN()

        if (epoch >= 1):
            try:
                weights = copy.deepcopy(self.personal_weights[round_it][epoch - 1])
#                 intercepts = copy.deepcopy(self.personal_intercepts[round_it][epoch - 1])
            except:
                print("Node: ", self.client_id)
                print("Epoch: ", epoch-1)
                print("Round: ", round_it)
                time.sleep(3)
                weights = copy.deepcopy(self.personal_weights[round_it][epoch - 1])
#                 intercepts = copy.deepcopy(self.personal_intercepts[round_it][epoch - 1])
                
        else:
            if(round_it>=1):
                try:
                    weights = copy.deepcopy(self.federated_weights[round_it - 1])
#                     intercepts = copy.deepcopy(self.federated_intercepts[round_it - 1])
#                     print("Federated weights set for epoch ", epoch, " in round  ", round_it)
                except: 
                    rounds_trained = len(self.nb_weights)
                    while(rounds_trained<round_it):
                        self.get_nb_weights(rounds_trained)
                        rounds_trained+=1
                    
                    try:
                        weights = copy.deepcopy(self.federated_weights[round_it - 1])
#                         intercepts = copy.deepcopy(self.federated_intercepts[round_it - 1])
                    except:
#                         print("Round: ", round_it)
#                         print("Node: ", self.client_id)
#                         print("Maxi var: ", maxi)
                        weights = copy.deepcopy(self.federated_weights[round_it - 1])
#                         intercepts = copy.deepcopy(self.federated_intercepts[round_it - 1])
            else:
                weights = None
#                 intercepts = None
                
#         lr.fit(X, y, coef_init=weights, intercept_init=intercepts)
        if(weights!=None):
            for layeri in range(len(model.layers)):
                (model.layers[layeri]).set_weights(weights[layeri])
#         print(X.shape)
#         print(y.shape)
        model.fit(X, y, verbose=0, epochs=1, steps_per_epoch = 1)
        local_weights = [layer.get_weights() for layer in model.layers]
#         local_intercepts = lr.intercept_
        
        if(epoch==(len(self.client_datasets) - 1)):
            
            X = self.test_dataset.drop('y', axis=1)
            y = self.test_dataset['y']
            X, y = self.data_preprocessing(X, y)
            
            acc = model.evaluate(X, y, verbose=0, return_dict=True)
            self.personal_accuracy[round_it] = acc['accuracy']
            
        return local_weights
        
#     def federated_averaging(self):
        

In [11]:
class AllClients:
    def __init__(self, clients):
        self.clients = clients
        self.training = {int(client): False for client in range(CLIENTS)}
        with open('training.json', 'w') as f:
            json.dump(self.training, f)

In [12]:
client_li = []
for i in range(CLIENTS):
    ag = Client(i, CLIENT_GRAPH[i], df_di[i], '.ipfs_fl'+str(i), BLKCHAIN_ACCOUNTS[i], test)
    client_li.append(ag)

allCl = AllClients(client_li)

In [15]:
allCl.clients[0].network_train_caller(1)

Training round:  0


2022-02-26 22:32:44.821728: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-26 22:32:44.822386: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-26 22:32:48.316703: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-26 22:32:48.316786: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-26 22:32:48.317278: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-K234JIGI): /proc/driver/nvidia/version does not exist
2022-02-26 22:32:48.320139: I tensorflow/core/platform/cpu_fe

Validation accuracy:  0.924047589302063



In [18]:
allCl.clients[0].get_nb_weights(1)

In [19]:
print(allCl.clients[0].personal_accuracy, allCl.clients[1].personal_accuracy, allCl.clients[4].personal_accuracy)

{0: 0.924047589302063, 1: 0.8807142972946167} {0: 0.9214285612106323, 1: 0.8761904835700989} {0: 0.9352381229400635, 1: 0.9357143044471741}


In [ ]:
allCl.clients[0].get_nb_weights(2)

In [19]:
def printLengths(clients, round_it):
    for i in range(len(clients.clients)):
        try:
            print(i,' ',len(clients.clients[i].personal_weights[round_it]))
        except:
            print(i,'  0')

In [15]:
allCl.clients[0].get_nb_weights(1)
printLengths(allCl, 0)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1Training node  2

Training node  0
Federated weights set for epoch  0  in round   1
Current Node:  2
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Training node  3
7/7 - 2s - loss: 2.0376 - accuracy: 0.3545 - 2s/epoch - 327ms/step
7/7 - 2s - loss: 2.3071 - accuracy: 0.0909 - 2s/epoch - 326ms/step
7/7 - 2s - loss: 2.3012 - accuracy: 0.1091 - 2s/epoch - 323ms/step
7/7 - 1s - loss: 1.4740 - accuracy: 0.5364 - 1s/epoch - 202ms/step
7/7 - 1s - loss: 1.9105 - accuracy: 0.3182 - 1s/epoch - 201ms/step
7/7 - 1s - loss: 1.3009 - accuracy: 0.6773 - 1s/epoch - 206ms/step
7/7 - 1s - loss: 1.5050 - accuracy: 0.5955 - 1s/epoch - 209ms/step
7/7 - 2s - loss: 0.8874 - accuracy: 0.7455 - 2s/epoch - 221ms/step
7/7 - 2s - loss: 1.1440 - accuracy: 0.6773 - 2s/epoch - 224ms/step
7/7 - 2s - loss: 0.6139 - accuracy: 0.7818 - 2s

 18.94 MiB / 18.94 MiB  100.00%

132/132 [==============================] - 9s 65ms/step - loss: 0.3767 - accuracy: 0.8938


7/7 - 2s - loss: 1.9183 - accuracy: 0.3545 - 2s/epoch - 356ms/step
7/7 - 2s - loss: 1.9168 - accuracy: 0.3636 - 2s/epoch - 218ms/step
7/7 - 1s - loss: 1.3001 - accuracy: 0.5955 - 1s/epoch - 208ms/step
7/7 - 1s - loss: 1.5101 - accuracy: 0.6318 - 1s/epoch - 194ms/step
7/7 - 2s - loss: 1.1564 - accuracy: 0.7455 - 2s/epoch - 217ms/step
7/7 - 1s - loss: 1.4321 - accuracy: 0.6727 - 1s/epoch - 210ms/step
7/7 - 1s - loss: 0.7756 - accuracy: 0.7682 - 1s/epoch - 201ms/step
7/7 - 1s - loss: 0.8774 - accuracy: 0.7545 - 1s/epoch - 198ms/step
7/7 - 1s - loss: 0.5327 - accuracy: 0.8318 - 1s/epoch - 201ms/step
7/7 - 1s - loss: 0.8391 - accuracy: 0.7591 - 1s/epoch - 196ms/step
132/132 [==============================] - 8s 58ms/step - loss: 0.4422 - accuracy: 0.8681


0   0
1   0
2   0
3   0


In [27]:
allCl.clients[0].get_nb_weights(2)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Federated weights set for epoch  0  in round   2
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Training node  3
Current Node:  2
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]


7/7 - 6s - loss: 0.8777 - accuracy: 0.7864 - 6s/epoch - 926ms/step
7/7 - 2s - loss: 2.0565 - accuracy: 0.2500 - 2s/epoch - 331ms/step
7/7 - 2s - loss: 1.9000 - accuracy: 0.3091 - 2s/epoch - 353ms/step
7/7 - 2s - loss: 0.7940 - accuracy: 0.7864 - 2s/epoch - 266ms/step
7/7 - 2s - loss: 1.5178 - accuracy: 0.6227 - 2s/epoch - 215ms/step
7/7 - 2s - loss: 0.6637 - accuracy: 0.7773 - 2s/epoch - 219ms/step
7/7 - 2s - loss: 1.5282 - accuracy: 0.6545 - 2s/epoch - 220ms/step
7/7 - 1s - loss: 0.4344 - accuracy: 0.8636 - 1s/epoch - 198ms/step
7/7 - 1s - loss: 0.8900 - accuracy: 0.7727 - 1s/epoch - 208ms/step
7/7 - 1s - loss: 0.5217 - accuracy: 0.8364 - 1s/epoch - 204ms/step
127/132 [===========================>..] - ETA: 0s - loss: 0.3800 - accuracy: 0.8974

 18.91 MiB / 18.91 MiB  100.00%

132/132 [==============================] - 8s 63ms/step - loss: 0.3810 - accuracy: 0.8971


7/7 - 3s - loss: 0.7442 - accuracy: 0.8045 - 3s/epoch - 436ms/step
7/7 - 2s - loss: 1.0012 - accuracy: 0.7682 - 2s/epoch - 221ms/step
7/7 - 1s - loss: 0.6652 - accuracy: 0.7636 - 1s/epoch - 205ms/step
7/7 - 1s - loss: 0.8052 - accuracy: 0.7545 - 1s/epoch - 209ms/step
7/7 - 2s - loss: 0.7071 - accuracy: 0.8045 - 2s/epoch - 226ms/step
7/7 - 2s - loss: 0.6094 - accuracy: 0.8227 - 2s/epoch - 216ms/step
7/7 - 2s - loss: 0.4883 - accuracy: 0.8636 - 2s/epoch - 225ms/step
7/7 - 2s - loss: 0.8036 - accuracy: 0.7909 - 2s/epoch - 226ms/step
7/7 - 2s - loss: 0.6743 - accuracy: 0.8136 - 2s/epoch - 215ms/step
7/7 - 1s - loss: 0.7480 - accuracy: 0.7955 - 1s/epoch - 194ms/step
132/132 [==============================] - 9s 67ms/step - loss: 0.2678 - accuracy: 0.9257


In [30]:
allCl.clients[0].personal_accuracy

{0: 0.8502380847930908, 1: 0.897857129573822, 2: 0.9233333468437195}

In [28]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   0

0   0
1   0
2   0
3   0


In [31]:
allCl.clients[0].get_nb_weights(3)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Training node  3
Current Node:  2
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Federated weights set for epoch  0  in round   3
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]


 0 B / 18.94 MiB    0.00%

Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]


7/7 - 7s - loss: 0.4528 - accuracy: 0.8682 - 7s/epoch - 966ms/step
7/7 - 2s - loss: 0.8828 - accuracy: 0.7682 - 2s/epoch - 337ms/step
7/7 - 2s - loss: 1.0050 - accuracy: 0.7409 - 2s/epoch - 352ms/step
7/7 - 2s - loss: 0.4290 - accuracy: 0.8818 - 2s/epoch - 283ms/step
7/7 - 2s - loss: 0.7448 - accuracy: 0.7818 - 2s/epoch - 226ms/step
7/7 - 2s - loss: 0.6182 - accuracy: 0.8591 - 2s/epoch - 226ms/step
7/7 - 2s - loss: 0.6072 - accuracy: 0.7864 - 2s/epoch - 219ms/step
7/7 - 2s - loss: 0.4198 - accuracy: 0.8773 - 2s/epoch - 221ms/step
7/7 - 2s - loss: 0.6603 - accuracy: 0.8182 - 2s/epoch - 220ms/step
7/7 - 2s - loss: 0.3684 - accuracy: 0.8727 - 2s/epoch - 235ms/step
132/132 [==============================] - 9s 67ms/step - loss: 0.2467 - accuracy: 0.9257


7/7 - 2s - loss: 0.4354 - accuracy: 0.8500 - 2s/epoch - 243ms/step
7/7 - 2s - loss: 0.4928 - accuracy: 0.8682 - 2s/epoch - 267ms/step
7/7 - 2s - loss: 0.4787 - accuracy: 0.8409 - 2s/epoch - 216ms/step
7/7 - 2s - loss: 0.5255 - accuracy: 0.8773 - 2s/epoch - 218ms/step
7/7 - 1s - loss: 0.4260 - accuracy: 0.8773 - 1s/epoch - 211ms/step
7/7 - 1s - loss: 0.4389 - accuracy: 0.8864 - 1s/epoch - 210ms/step
7/7 - 1s - loss: 0.5033 - accuracy: 0.8500 - 1s/epoch - 209ms/step
7/7 - 1s - loss: 0.2254 - accuracy: 0.9182 - 1s/epoch - 208ms/step
7/7 - 1s - loss: 0.4541 - accuracy: 0.8636 - 1s/epoch - 210ms/step
132/132 [==============================] - 9s 68ms/step - loss: 0.2469 - accuracy: 0.9319


In [32]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   0


In [35]:
allCl.clients[0].personal_accuracy, allCl.clients[1].personal_accuracy, allCl.clients[2].personal_accuracy, allCl.clients[3].personal_accuracy

({0: 0.8502380847930908,
  1: 0.897857129573822,
  2: 0.9233333468437195,
  3: 0.9366666674613953},
 {0: 0.7840476036071777,
  1: 0.8845238089561462,
  2: 0.9295238256454468,
  3: 0.9059523940086365},
 {0: 0.8678571581840515,
  1: 0.8680952191352844,
  2: 0.9257143139839172,
  3: 0.9319047331809998},
 {0: 0.8938094973564148, 1: 0.8971428275108337, 2: 0.9257143139839172})

In [36]:
allCl.clients[0].get_nb_weights(4)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  Training node  0
2
Current Node:  1
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 1]
Training node  3
Federated weights set for epoch  0  in round   4
Current Node:  2
Untrained Neighbour:  [3]
All neighbours:  [0, 3, 2]
Training node  3
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]
Current Node:  3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]


7/7 - 7s - loss: 0.3082 - accuracy: 0.9182 - 7s/epoch - 1s/step
7/7 - 2s - loss: 0.5756 - accuracy: 0.8591 - 2s/epoch - 313ms/step
7/7 - 2s - loss: 0.4967 - accuracy: 0.8636 - 2s/epoch - 320ms/step
7/7 - 2s - loss: 0.2516 - accuracy: 0.9136 - 2s/epoch - 239ms/step
7/7 - 1s - loss: 0.4746 - accuracy: 0.8500 - 1s/epoch - 210ms/step
7/7 - 1s - loss: 0.3600 - accuracy: 0.8955 - 1s/epoch - 201ms/step
7/7 - 1s - loss: 0.4142 - accuracy: 0.8545 - 1s/epoch - 209ms/step
7/7 - 1s - loss: 0.3205 - accuracy: 0.9045 - 1s/epoch - 208ms/step
7/7 - 1s - loss: 0.5407 - accuracy: 0.8545 - 1s/epoch - 207ms/step
7/7 - 1s - loss: 0.3113 - accuracy: 0.9227 - 1s/epoch - 211ms/step
131/132 [============================>.] - ETA: 0s - loss: 0.2191 - accuracy: 0.9363

 18.84 MiB / 18.84 MiB  100.00%

132/132 [==============================] - 9s 68ms/step - loss: 0.2188 - accuracy: 0.9364


7/7 - 1s - loss: 0.3686 - accuracy: 0.9091 - 1s/epoch - 210ms/step
7/7 - 2s - loss: 0.3782 - accuracy: 0.9182 - 2s/epoch - 219ms/step
7/7 - 1s - loss: 0.3726 - accuracy: 0.8682 - 1s/epoch - 199ms/step
7/7 - 1s - loss: 0.3238 - accuracy: 0.8955 - 1s/epoch - 206ms/step
7/7 - 1s - loss: 0.3119 - accuracy: 0.9045 - 1s/epoch - 200ms/step
7/7 - 1s - loss: 0.4294 - accuracy: 0.8682 - 1s/epoch - 208ms/step
7/7 - 2s - loss: 0.2495 - accuracy: 0.9091 - 2s/epoch - 220ms/step
7/7 - 2s - loss: 0.3801 - accuracy: 0.8955 - 2s/epoch - 224ms/step
7/7 - 1s - loss: 0.2890 - accuracy: 0.9182 - 1s/epoch - 198ms/step
7/7 - 1s - loss: 0.3716 - accuracy: 0.8864 - 1s/epoch - 206ms/step
132/132 [==============================] - 7s 56ms/step - loss: 0.1861 - accuracy: 0.9462


In [37]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)
print()
printLengths(allCl, 4)

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   5

0   5
1   5
2   5
3   0


In [38]:
allCl.clients[0].personal_accuracy, allCl.clients[1].personal_accuracy, allCl.clients[2].personal_accuracy, allCl.clients[3].personal_accuracy

({0: 0.8502380847930908,
  1: 0.897857129573822,
  2: 0.9233333468437195,
  3: 0.9366666674613953,
  4: 0.9504761695861816},
 {0: 0.7840476036071777,
  1: 0.8845238089561462,
  2: 0.9295238256454468,
  3: 0.9059523940086365,
  4: 0.9461904764175415},
 {0: 0.8678571581840515,
  1: 0.8680952191352844,
  2: 0.9257143139839172,
  3: 0.9319047331809998,
  4: 0.9459523558616638},
 {0: 0.8938094973564148,
  1: 0.8971428275108337,
  2: 0.9257143139839172,
  3: 0.9364285469055176})

In [25]:
allCl.clients[0].get_nb_weights(5)

Current Node:  0
Untrained Neighbour:  [1, 2, 0]
All neighbours:  [1, 2, 0]
Training node  1
Training node  2
Training node  0
Current Node:  1
Current Node:  2
Untrained Neighbour: Untrained Neighbour:  [3]
All neighbours:   [3]
All neighbours:  [0, 3, 2]
[0, 3, 1]
Training node  3
Training node  3
Current Node: Current Node:   3
Untrained Neighbour:  []
All neighbours:  [1, 2, 3]3
Untrained Neighbour:  []
All neighbours:  
[1, 2, 3]
Training finished for  0


 358 B / 358 B  100.00%

Training Caller 2:  0
Training finished for  3


 355 B / 355 B  100.00%

Training Caller 2:  3


 354 B / 354 B  100.00%

Training finished for  1
Training Caller 2:  1
Training over
Client  3  training already.
Training Caller 2:  3


 360 B / 360 B  100.00%

Training finished for  2
Training Caller 2:  2


In [27]:
printLengths(allCl, 0)
print()
printLengths(allCl, 1)
print()
printLengths(allCl, 2)
print()
printLengths(allCl, 3)
print()
printLengths(allCl, 4)
print()
printLengths(allCl, 5)

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   10

0   10
1   10
2   10
3   0


In [27]:
allCl.clients[0].federated_intercepts

{0: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159]),
 1: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159]),
 2: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159]),
 3: array([-4.43722862, -2.47253855,  2.71434389, -0.83139894, -4.84719159])}

In [24]:
allCl.clients[3].personal_accuracy

{0: 0.19}

In [17]:
allCl.clients[0].nb_intercepts[0]

[[-9.08828927688947,
  -1.897113601450453,
  3.324919978719681,
  -3.607171033290969,
  -5.6914308345434925],
 [-2.7825014532674115,
  -1.9163357277199058,
  1.4142788785330778,
  7.635749088239045,
  -9.203838156493957],
 [-1.4408951288098966,
  -3.6041663195640137,
  3.4038328221333707,
  -6.522774884794829,
  0.35369420967760234]]

In [20]:
allCl.clients[0].nb_intercepts[1]

[[-9.08828927688947,
  -1.897113601450453,
  3.324919978719681,
  -3.607171033290969,
  -5.6914308345434925],
 [-2.7825014532674115,
  -1.9163357277199058,
  1.4142788785330778,
  7.635749088239045,
  -9.203838156493957],
 [-1.4408951288098966,
  -3.6041663195640137,
  3.4038328221333707,
  -6.522774884794829,
  0.35369420967760234]]

In [10]:
def client_computation_caller(inp):
    client_instance, body = inp
    return_message = client_instance.train_client(body['round'])
    return return_message

In [11]:
for ro in range(ROUNDS):
    m = multiprocessing.Manager()
    lock = m.Lock()
    with ThreadPool(CLIENTS) as calling_pool:
        args = []
        for cl in range(CLIENTS):
            body = {'round': ro, 'lock': lock}
            args.append((allCl.clients[cl], body))
        calling_pool.map(client_computation_caller, args)

In [46]:
allCl.clients[1].personal_intercepts[0]

[array([-1.19024283, -8.22172283, -9.42661473, -3.97311779, -4.33640538]),
 array([-10.33395532,   3.24845949,   5.356186  ,  -9.3852317 ,
         -2.30635674]),
 array([-10.55145539, -12.46190679,   0.64291909,  -1.13745339,
         -0.18818607]),
 array([  0.52149334, -16.64293375,  -6.27774062,  -0.99584345,
         -0.70731457]),
 array([-1.44089513, -3.60416632,  3.40383282, -6.52277488,  0.35369421])]